In [205]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np 

In [206]:
url='https://www.mscorecard.com/mscorecard/showcourse.php?cid=141521162939611337'
#url='https://www.mscorecard.com/mscorecard/showcourse.php?cid=14152075278110428'

In [207]:
#dfs = pd.read_html('https://www.mscorecard.com/mscorecard/showcourse.php?cid=141521162939611337')
df = pd.read_html(url)[0]

In [208]:
#dfs

In [209]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)

In [210]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [211]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Test for the number of holes
if (len(tr_elements)) == 22:
    h=['2','3','4','5','6','7','8','9','10']
if (len(tr_elements)) == 33:
    h=['2','3','4','5','6','7','8','9','10','12','13','14','15','16','17','18','19','20']

In [212]:
#print (tr_elements)

In [213]:
#Check the length of the first 12 rows
#[len(T) for T in tr_elements[:22]]

In [214]:
#Create empty list
tees=[]
yards=[]
holes=[]
col=[]
par=[]
w_par=[]
m_hcp=[]
w_hcp=[]
m_slope=[]
m_rating=[]
w_slope=[]
w_rating=[]
i=0
j=2

In [215]:
#For each row, store each first element (header) and an empty list
for t in tr_elements[1]:
    i+=1
    name=t.text_content()
    #print ('%d:"%s"'%(i,name))
    #print(name)
    if name not in ['Hole','Par','Hcp']:
        tees.append(name)
    col.append((name,[]))
print('Tees', tees)

Tees ['Black', 'Blue', 'White', 'Red (L)']
4


In [226]:
#  front 9 and back 9 yardages
def get_value(index):
    if (len(tr_elements)) == 22:
        return list(np.concatenate([df.iloc[:,index][2:11].to_numpy()]))
    if (len(tr_elements)) == 33:
        return list(np.concatenate([df.iloc[:,index][2:11].to_numpy(), df.iloc[:,index][12:21].to_numpy()]))
yards = []
for i in range(3,len(tees)+3):
    yards.append(get_value(i))
print('Yards', yards) 

Yards [['451', '371', '200', '346', '442', '419', '241', '385', '541', '561', '387', '308', '456', '180', '584', '181', '405', '165'], ['401', '354', '164', '298', '408', '398', '210', '319', '511', '538', '372', '288', '448', '149', '560', '160', '379', '148'], ['346', '326', '152', '291', '363', '388', '200', '260', '415', '511', '361', '255', '386', '138', '548', '149', '330', '111'], ['251', '277', '117', '261', '325', '354', '166', '225', '355', '439', '278', '187', '325', '94', '489', '111', '269', '90']]
test 387


In [218]:
# front 9 and back 9 MEN pars and handicaps
for x in h:
    for t in (tr_elements[int(x)]):
        name=t.text_content()
        holes.append(name)
    par.append(holes[1])
    m_hcp.append(holes[2])
    holes.clear()
print('Men par', par)
print('Men Handicap', m_hcp)

Men par ['4', '4', '3', '4', '4', '4', '3', '4', '5', '5', '4', '4', '4', '3', '5', '3', '4', '3']
Men Handicap ['2', '10', '18', '14', '6', '8', '16', '12', '4', '3', '9', '11', '5', '15', '1', '13', '7', '17']


In [219]:
# men slope
for t in (tr_elements[23]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    m_slope.append(holes[i])
holes.clear()
print('Men slope', m_slope)

Men slope ['142', '135', '130', '-']


In [220]:
# men rating
for t in (tr_elements[24]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    m_rating.append(holes[i])
holes.clear()
print('Men rating', m_rating)

Men rating ['71.8', '69.3', '66.7', '-']


In [221]:
# women slope
for t in (tr_elements[25]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    w_slope.append(holes[i])
holes.clear()
print('Women slope', w_slope)

Women slope ['-', '-', '134', '118']


In [222]:
# women rating
for t in (tr_elements[26]):
    name=t.text_content()
    holes.append(name)
for i in range(len(tees)):
    i+=1
    w_rating.append(holes[i])
holes.clear()
print('Women rating',  w_rating)

Women rating ['-', '-', '72.2', '67']


In [475]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [425]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [355]:
df.head()

,Hole,Par,Hcp,Black,Blue,White,Red (L)
